In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder 

In [2]:
import torch
import torchvision
import torch.nn.functional as F
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader 

In [3]:
from sklearn import datasets
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [4]:
pip install -U sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [5]:
from sentence_transformers import SentenceTransformer
model_transformer = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [6]:
#read all csv data
data_set_train = pd.read_csv("gs_train.csv")
data_set_val = pd.read_csv("gs_val.csv")
data_set_test = pd.read_csv("gs_test.csv")
data_set_amazon = pd.read_csv("1_amazon.csv")
data_set_google = pd.read_csv("2_google.csv")
dataset_train = data_set_train.iloc[:,:].values
dataset_val = data_set_val.iloc[:,:].values
dataset_test = data_set_test.iloc[:,:].values
dataset_amazon = data_set_amazon.iloc[:,:].values
dataset_google = data_set_google.iloc[:,:].values

In [7]:
#create data frame for the five initial data
df_train = pd.DataFrame({'id_amazon': dataset_train[:,0],
                   'id_google': dataset_train[:,1],
                   'matching': dataset_train[:,2]  })
df_val = pd.DataFrame({'id_amazon': dataset_val[:,0],
                   'id_google': dataset_val[:,1],
                   'matching': dataset_val[:,2]  })
df_test = pd.DataFrame({'id_amazon': dataset_test[:,0],
                   'id_google': dataset_test[:,1],
                   'matching': dataset_test[:,2]  })

df_amazon = pd.DataFrame({'id': dataset_amazon[:,0],
                   'name': dataset_amazon[:,1],
                   'description': dataset_amazon[:,2],
                   'manufacturer': dataset_amazon[:,3],
                   'price': dataset_amazon[:,4]  })
df_google = pd.DataFrame({'id': dataset_google[:,0],
                   'name': dataset_google[:,1],
                   'description': dataset_google[:,2],
                   'manufacturer': dataset_google[:,3],
                   'price': dataset_google[:,4]  })

In [8]:
#join the data to a big dataframe to ease future work
join_train_amazon = df_train.join(df_amazon.set_index('id'), on='id_amazon', lsuffix='_caller', rsuffix='_other')
join_train_google = df_train.join(df_google.set_index('id'), on='id_google', lsuffix='_caller', rsuffix='_other')
join_train_amazon_google = join_train_amazon.join(df_google.set_index('id'), on='id_google', lsuffix='_caller', rsuffix='_other')

join_val_amazon = df_val.join(df_amazon.set_index('id'), on='id_amazon', lsuffix='_caller', rsuffix='_other')
join_val_google = df_val.join(df_google.set_index('id'), on='id_google', lsuffix='_caller', rsuffix='_other')
join_val_amazon_google = join_val_amazon.join(df_google.set_index('id'), on='id_google', lsuffix='_caller', rsuffix='_other')

join_test_amazon = df_test.join(df_amazon.set_index('id'), on='id_amazon', lsuffix='_caller', rsuffix='_other')
join_test_google = df_test.join(df_google.set_index('id'), on='id_google', lsuffix='_caller', rsuffix='_other')
join_test_amazon_google = join_test_amazon.join(df_google.set_index('id'), on='id_google', lsuffix='_caller', rsuffix='_other')
#shape test
join_train_amazon_google

,id_amazon,id_google,matching,name_caller,description_caller,manufacturer_caller,price_caller,name_other,description_other,manufacturer_other,price_other
0,b0001wn1ee,http://www.google.com/base/feeds/snippets/9255...,False,microsoft powerpoint 2004 (mac),powerpoint 2004 offers you the powerful advanc...,microsoft,229.99,microsoft office outlook 2007,key features: manages time and info works with...,NaN,109.95
1,b000g80lku,http://www.google.com/base/feeds/snippets/1531...,False,peachtree by sage premium accounting 2007 - ac...,peachtree premium accounting 2007 accountants'...,sage software,499.99,sage software peachtree pro accounting 2007 so...,peachtree pro accounting 2007 software for win...,NaN,159.95
2,b000070ilx,http://www.google.com/base/feeds/snippets/1575...,False,detto move2mac usb,move2mac is a complete solution for moving inf...,detto technologies inc,71.31,punch software 25100 - punch! master landscape...,punch software 25100 : punch! master landscape...,NaN,45.97
3,b000ub6avc,http://www.google.com/base/feeds/snippets/9526...,False,adobe cs3 design standard macosx dvd,design std cs3 3 mac ed 1u,adobe,399.0,adobe premiere pro cs3,multicore intel� processor (adobe� onlocation?...,NaN,825.99
4,b000cc22ys,http://www.google.com/base/feeds/snippets/1024...,False,microsoft mappoint 2006 with gps,microsoft mappoint 2006 helps business users t...,microsoft,349.0,microsoft onenote 2007 upgrade (pc),key features: professional content share your ...,NaN,79.95
...,...,...,...,...,...,...,...,...,...,...,...
5903,b000ndibqy,http://www.google.com/base/feeds/snippets/1751...,False,adobe photoshop cs3 extended upgrade [mac],note: this is the upgrade version of adobe pho...,adobe,349.0,adobe contribute cs3 upgrade,system requirements powerpc g4 or g5 or intel ...,NaN,82.99
5904,b00029bqa2,http://www.google.com/base/feeds/snippets/1766...,False,disney's 1st & 2nd grade bundle (pixar 1st gra...,disney's 1st & 2nd grade bundle will help your...,disney,14.99,micromat techtool protege utility with 1gb fir...,system requirements ppc g3 or better capable o...,NaN,219.99
5905,b000fvqo0o,http://www.google.com/base/feeds/snippets/2209...,True,punch! professional home design platinum v10,punch! professional home design suite platinum...,punch! software,99.0,punch software professional home design platin...,punch! professional home design suite platinum...,NaN,99.99
5906,b000o27ti0,http://www.google.com/base/feeds/snippets/9761...,False,adobe creative suite cs3 design standard upgrade,note: this is the upgrade version of adobe cre...,adobe,399.0,adobe indesign cs3 for mac upgrade,system requirements powerpc g4 or g5 or intel ...,NaN,205.99


In [9]:
#make sure that string is not mixed with decimals
for j in range (3,11):  
    for i in range(len(join_train_amazon_google.iloc[:,j])):
        join_train_amazon_google.iloc[i,j] = str(join_train_amazon_google.iloc[i,j])
    for i in range(len(join_val_amazon_google.iloc[:,j])):
        join_val_amazon_google.iloc[i,j] = str(join_val_amazon_google.iloc[i,j])
    for i in range(len(join_test_amazon_google.iloc[:,j])):
        join_test_amazon_google.iloc[i,j] = str(join_test_amazon_google.iloc[i,j])

In [10]:
#use sentence transformers to extract the embedding of all names, descriptions and manufacturers
embeddings_train_amazon_name = model_transformer.encode(join_train_amazon_google.iloc[:,3])
embeddings_train_amazon_description = model_transformer.encode(join_train_amazon_google.iloc[:,4])
embeddings_train_amazon_manufacturer = model_transformer.encode(join_train_amazon_google.iloc[:,5])
embeddings_train_amazon = (embeddings_train_amazon_name+embeddings_train_amazon_description+embeddings_train_amazon_manufacturer)/3

embeddings_train_google_name = model_transformer.encode(join_train_amazon_google.iloc[:,7])
embeddings_train_google_description = model_transformer.encode(join_train_amazon_google.iloc[:,8])
embeddings_train_google_manufacturer = model_transformer.encode(join_train_amazon_google.iloc[:,9])
embeddings_train_google = (embeddings_train_google_name+embeddings_train_google_description+embeddings_train_google_manufacturer)/3

In [11]:
#Same for validation and test
embeddings_val_amazon_name = model_transformer.encode(join_val_amazon_google.iloc[:,3])
embeddings_val_amazon_description = model_transformer.encode(join_val_amazon_google.iloc[:,4])
embeddings_val_amazon_manufacturer = model_transformer.encode(join_val_amazon_google.iloc[:,5])
embeddings_val_amazon = (embeddings_val_amazon_name+embeddings_val_amazon_description+embeddings_val_amazon_manufacturer)/3

embeddings_val_google_name = model_transformer.encode(join_val_amazon_google.iloc[:,7])
embeddings_val_google_description = model_transformer.encode(join_val_amazon_google.iloc[:,8])
embeddings_val_google_manufacturer = model_transformer.encode(join_val_amazon_google.iloc[:,9])
embeddings_val_google = (embeddings_val_google_name+embeddings_val_google_description+embeddings_val_google_manufacturer)/3

In [12]:
#Same for validation and test
embeddings_test_amazon_name = model_transformer.encode(join_test_amazon_google.iloc[:,3])
embeddings_test_amazon_description = model_transformer.encode(join_test_amazon_google.iloc[:,4])
embeddings_test_amazon_manufacturer = model_transformer.encode(join_test_amazon_google.iloc[:,5])
embeddings_test_amazon = (embeddings_test_amazon_name+embeddings_test_amazon_description+embeddings_test_amazon_manufacturer)/3

embeddings_test_google_name = model_transformer.encode(join_test_amazon_google.iloc[:,7])
embeddings_test_google_description = model_transformer.encode(join_test_amazon_google.iloc[:,8])
embeddings_test_google_manufacturer = model_transformer.encode(join_test_amazon_google.iloc[:,9])
embeddings_test_google = (embeddings_test_google_name+embeddings_test_google_description+embeddings_test_google_manufacturer)/3

In [13]:
#Price is also transferred to string to achieve high score 
embeddings_train_amazon_price = model_transformer.encode(join_train_amazon_google.iloc[:,6])
embeddings_train_google_price = model_transformer.encode(join_train_amazon_google.iloc[:,10])
embeddings_val_amazon_price = model_transformer.encode(join_val_amazon_google.iloc[:,6])
embeddings_val_google_price = model_transformer.encode(join_val_amazon_google.iloc[:,10])
embeddings_test_amazon_price = model_transformer.encode(join_test_amazon_google.iloc[:,6])
embeddings_test_google_price = model_transformer.encode(join_test_amazon_google.iloc[:,10])

In [14]:
embeddings_train_amazon_description

array([[-0.3318212 , -0.21919611, -0.3129619 , ...,  0.06168164,
        -0.08299212,  0.12433101],
       [-0.20135523, -0.1138659 , -0.5085314 , ..., -0.44308835,
         0.07250084,  0.12026882],
       [-0.06041463, -0.51194274,  0.07474437, ...,  0.47617748,
        -0.56341755,  0.05152213],
       ...,
       [-0.44108367,  0.41207567, -0.09806831, ...,  0.02787065,
        -0.11102119, -0.00209533],
       [-0.3483483 , -0.09811661, -0.0481395 , ..., -0.2462284 ,
         0.04623684,  0.04324549],
       [ 0.10170409, -0.23083246, -0.25402752, ...,  0.30123457,
         0.4638311 , -0.12120333]], dtype=float32)

In [15]:
label_encoder = LabelEncoder() 

In [16]:
#to tensor
x_train = np.hstack((embeddings_train_amazon_name, embeddings_train_amazon_description, embeddings_train_amazon_manufacturer, embeddings_train_amazon_price, embeddings_train_google_name, embeddings_train_google_description, embeddings_train_google_manufacturer, embeddings_train_google_price))
x_train = np.vstack(x_train).astype(np.float32)
x_train = torch.from_numpy(x_train).to(torch.float32)
x_val = np.hstack((embeddings_val_amazon_name, embeddings_val_amazon_description, embeddings_val_amazon_manufacturer, embeddings_val_amazon_price, embeddings_val_google_name, embeddings_val_google_description, embeddings_val_google_manufacturer, embeddings_val_google_price))
x_val = np.vstack(x_val).astype(np.float32)
x_val = torch.from_numpy(x_val).to(torch.float32)
x_test = np.hstack((embeddings_test_amazon_name, embeddings_test_amazon_description, embeddings_test_amazon_manufacturer, embeddings_test_amazon_price, embeddings_test_google_name, embeddings_test_google_description, embeddings_test_google_manufacturer, embeddings_test_google_price))
x_test = np.vstack(x_test).astype(np.float32)
x_test = torch.from_numpy(x_test).to(torch.float32)

y_train = join_train_amazon_google.iloc[:,2]  # dataframe: true or false
y_train = label_encoder.fit_transform(y_train) # np array: 1 or 2
y_train = torch.from_numpy(y_train).to(torch.float32) # torch tensor: 1 or 2
y_val = join_val_amazon_google.iloc[:,2]
y_val = label_encoder.fit_transform(y_val)
y_val = torch.from_numpy(y_val).to(torch.float32)
y_test = join_test_amazon_google.iloc[:,2]
y_test = label_encoder.fit_transform(y_test)
y_test = torch.from_numpy(y_test).to(torch.float32)

In [17]:
x_train

tensor([[-0.1517, -0.5100, -0.0079,  ..., -0.4089,  0.1780, -0.1760],
        [ 0.0756, -0.2756, -0.5819,  ..., -0.2340,  0.4008, -0.1342],
        [-0.2017,  0.1395, -0.1800,  ..., -0.2745,  0.2544,  0.0432],
        ...,
        [-0.6347,  0.5674, -0.1356,  ..., -0.5849,  0.4540,  0.3243],
        [-0.6103,  0.0489, -0.0089,  ..., -0.4281,  0.3704, -0.2706],
        [-0.2779, -0.7731, -0.4609,  ..., -0.3726,  0.1451, -0.1696]])

In [18]:
class Our_Model(nn.Module):
    def __init__(self, activation=nn.Sigmoid(), input_size=2, hidden_size=50, reg = 0):
        super(Our_Model, self).__init__() 
        
        self.activation = activation
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.reg_strength = reg
        
        self.pack1 = nn.Sequential(nn.Linear(1536,40),#L1
                                   self.activation,
                                   nn.Linear(40, 40),#L2
                                   nn.Dropout(),  
                                   nn.LeakyReLU(0.2),
                                   nn.Linear(40, 20), #3
                                   #nn.Dropout(0.5),
                                   self.activation,
                                   nn.Linear(20, 30),#L4
                                   nn.Dropout(),
                                   nn.LeakyReLU(0.2),
                                   nn.Linear(30,10),#L5
                                   self.activation,
                                   nn.Linear(10,10),#L6                                   
                                   self.activation,
                                   nn.Linear(10,1)#L8
                                  )        
    def forward(self, x):
        x = self.pack1(x)
        x = self.activation(x.squeeze())
        return x  

In [19]:
def initialize_weight(module):    
    if isinstance(module, (nn.Linear, nn.Conv2d)): 
        nn.init.xavier_normal_(module.weight)#,gain=nn.init.calculate_gain('relu'))  

In [20]:
class Combi_Loss(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, pre, label, alpha):    
        dice = (2*pre*label+1e-5) / (pre+label+1e-5)
        dice_loss = 1-dice
        bce_loss = ( -label * torch.log(pre) - (1 - label) * torch.log(1 - pre) )/len(label)
        loss = alpha*dice_loss + (1-alpha)*bce_loss 
        return loss
    def backward(self, pre, label, alpha):
        gradient = - alpha *( (2*label-1)*1e-5+2*label**2 ) /(pre+label+1e-5)**2 - (1-alpha)*(label/pre)/len(label) + (1-alpha)*(1-label)/(1-pre)/len(label)
        return gradient    

In [21]:
class L2(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, pre, label):    
        result = (pre - label)**2        
        return result
    def backward(self, pre, label):
        gradient = 2*(pre-label)
        return gradient 

In [22]:
#dataset
train_set = TensorDataset(x_train, y_train)
val_set = TensorDataset(x_val, y_val)
#dataloader
dataloaders = {}
dataloaders["train"] = DataLoader(train_set, batch_size = 16, shuffle=True)
dataloaders["val"] = DataLoader(val_set, batch_size = 16)

In [23]:
batchsize = 16
model = Our_Model(activation=nn.Sigmoid(), input_size=batchsize, hidden_size=50)
model.apply(initialize_weight)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0004)

In [24]:
loss_fn = Combi_Loss()
compute_L2 = L2()
loss_logistic = LogisticRegression()

In [25]:
def fit_train2(model, optimizer, num_epochs, batch_size=32, alpha = 0.5):
    overfit_index = 0
    min_loss = 20 
    for epoch in range(num_epochs):
        # Training
        model.train()                         
        for xb, yb in dataloaders["train"]:  
            L2_distance = compute_L2(xb[:,:1536],xb[:,1536:3072])
            logits = model(L2_distance)
            loss = loss_fn(logits, yb.squeeze(),alpha) 
            '''#add regulation
            l2_lambda = 5e-5
            l2_norm = sum(p.pow(2.0).sum() for p in model.parameters())
            loss = loss + l2_lambda*l2_norm
            '''
            model.zero_grad()
            loss.backward(torch.ones_like(loss)) ############
            optimizer.step() 
            
        # Evaluation
        model.eval()
        loss = 0
        with torch.no_grad():
            for xb, yb in dataloaders["val"]:
                L2_distance = compute_L2(xb[:,:1536],xb[:,1536:3072])
                logits = model(L2_distance)
                loss += loss_fn(logits, torch.squeeze(yb),alpha).sum()
            avg_loss = loss / len(dataloaders["val"])
            print(f"Epoch {epoch}: val_loss = {avg_loss:.3f}") 
            
            #early stopping
            if  min_loss < avg_loss:
                overfit_index = overfit_index+1
                if overfit_index>=3:
                    print("early stopping")
                    break 
            else:
                min_loss = avg_loss
                overfit_index = 0

In [26]:
fit_train2(model, optimizer, num_epochs = 40, batch_size=16,alpha = 0)

Epoch 0: val_loss = 0.439
Epoch 1: val_loss = 0.422
Epoch 2: val_loss = 0.329
Epoch 3: val_loss = 0.296
Epoch 4: val_loss = 0.277
Epoch 5: val_loss = 0.266
Epoch 6: val_loss = 0.261
Epoch 7: val_loss = 0.250
Epoch 8: val_loss = 0.240
Epoch 9: val_loss = 0.236
Epoch 10: val_loss = 0.252
Epoch 11: val_loss = 0.232
Epoch 12: val_loss = 0.234
Epoch 13: val_loss = 0.234
Epoch 14: val_loss = 0.232
Epoch 15: val_loss = 0.233
Epoch 16: val_loss = 0.236
Epoch 17: val_loss = 0.234
early stopping


In [27]:
L2_distance = compute_L2.forward(x_test[:,:1536],x_test[:,1536:3072])
y_pred = model.forward(L2_distance)
y_pred = ( y_pred > 0.5)
precision = precision_score(y_test.squeeze(), y_pred)
recall = recall_score(y_test.squeeze(), y_pred)
accuracy = accuracy_score(y_test, y_pred)
print('Precision: ',precision)
print('Recall: ',recall)
print('F1 Score: ',2*precision*recall/(precision+recall+0.0000001)  )
print('accuracy: ',accuracy)


Precision:  0.653179190751445
Recall:  0.8828125
F1 Score:  0.7508305159015937
accuracy:  0.9102870813397129
